In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv('sentiment_dataset.csv')  
print(f"Размер датасета: {df.shape}")
print(f"Распределение классов:\n{df['label'].value_counts()}")

Размер датасета: (290458, 3)
Распределение классов:
label
2    96992
1    96877
0    96589
Name: count, dtype: int64


In [20]:
df.head(5)

,text,label,src,cleaned_text
0,"Пальто красивое, но пришло с дырой в молнии. П...",0,rureviews,пальто красивое пришло дырой молнии просила вы...
1,"Очень долго шел заказ,ждала к новому году,приш...",0,rureviews,очень долго шел заказ ждала новому году пришел...
2,"Могу сказать одно, брюки нормальные, НО они бы...",0,rureviews,могу сказать одно брюки нормальные порваны мал...
3,"Доставка быстрая, меньше месяца. Заказывали ра...",0,rureviews,доставка быстрая меньше месяца заказывали разм...
4,Мне не очень понравилось это платье. Размер ...,0,rureviews,очень понравилось это платье размер l подошёл ...


In [5]:
import re
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nepip\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    
    text = text.lower()
    
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    
    russian_stopwords = stopwords.words('russian')
    words = text.split()
    words = [word for word in words if word not in russian_stopwords]
    
    return ' '.join(words).strip()

In [7]:
df['cleaned_text'] = df['text'].apply(preprocess_text)

In [8]:
vectorizer = TfidfVectorizer(
    max_features=5000,  
    min_df=2,           
    max_df=0.8,         
    ngram_range=(1, 2)  
)

In [9]:
X = vectorizer.fit_transform(df['cleaned_text'])
y = df['label']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Размер обучающей выборки: {X_train.shape}")
print(f"Размер тестовой выборки: {X_test.shape}")

Размер обучающей выборки: (232366, 5000)
Размер тестовой выборки: (58092, 5000)


In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression(
    random_state=42,
    max_iter=500,
    multi_class='multinomial',
    solver='sag',
    C=1.0,
    class_weight='balanced',
    penalty='l2',
    n_jobs=-1   
)

model.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=500,
                   multi_class='multinomial', n_jobs=-1, random_state=42,
                   solver='sag')

In [12]:
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)

In [13]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.6761


In [14]:
classes = ['нейтральный', 'позитивный', 'негативный']

for i, class_name in enumerate(classes):
    y_test_binary = (y_test == i)
    y_pred_binary = (y_pred == i)
    
    precision = precision_score(y_test_binary, y_pred_binary, zero_division=0)
    recall = recall_score(y_test_binary, y_pred_binary, zero_division=0)
    f1 = f1_score(y_test_binary, y_pred_binary, zero_division=0)
    
    print(f"\nКласс: {class_name}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")


Класс: нейтральный
Precision: 0.5703
Recall: 0.5875

Класс: позитивный
Precision: 0.7680
Recall: 0.7627

Класс: негативный
Precision: 0.6938
Recall: 0.6778


In [15]:
print(classification_report(y_test, y_pred, 
                          target_names=['нейтральный', 'позитивный', 'негативный']))

              precision    recall  f1-score   support

 нейтральный       0.57      0.59      0.58     19318
  позитивный       0.77      0.76      0.77     19375
  негативный       0.69      0.68      0.69     19399

    accuracy                           0.68     58092
   macro avg       0.68      0.68      0.68     58092
weighted avg       0.68      0.68      0.68     58092



In [17]:
def predict_sentiment(text, model, vectorizer):
    cleaned_text = preprocess_text(text)
    
    text_vector = vectorizer.transform([cleaned_text])
    
    prediction = model.predict(text_vector)[0]
    probabilities = model.predict_proba(text_vector)[0]
    
    class_names = {0: 'нейтральный', 1: 'позитивный', 2: 'негативный'}
    
    result = {
        'text': text[:100] + '...' if len(text) > 100 else text,
        'prediction': class_names[prediction],
        'probabilities': {
            'нейтральный': f"{probabilities[0]:.4f}",
            'позитивный': f"{probabilities[1]:.4f}",
            'негативный': f"{probabilities[2]:.4f}"
        }
    }
    
    return result

In [18]:
test_texts = [
    "Очень понравилось, прекрасное обслуживание!",
    "Средне, ничего особенного.",
    "Ужасный опыт, никогда больше не вернусь.",
    "В целом нормально, но есть недочёты.",
    "Супер! Рекомендую всем!"
]

for text in test_texts:
    result = predict_sentiment(text, model, vectorizer)
    print(f"\nТекст: {result['text']}")
    print(f"Предсказанный класс: {result['prediction']}")
    print(f"Вероятности: {result['probabilities']}")
    print("-"*40)


Текст: Очень понравилось, прекрасное обслуживание!
Предсказанный класс: позитивный
Вероятности: {'нейтральный': '0.0601', 'позитивный': '0.9344', 'негативный': '0.0054'}
----------------------------------------

Текст: Средне, ничего особенного.
Предсказанный класс: нейтральный
Вероятности: {'нейтральный': '0.8486', 'позитивный': '0.0361', 'негативный': '0.1153'}
----------------------------------------

Текст: Ужасный опыт, никогда больше не вернусь.
Предсказанный класс: негативный
Вероятности: {'нейтральный': '0.3007', 'позитивный': '0.0573', 'негативный': '0.6420'}
----------------------------------------

Текст: В целом нормально, но есть недочёты.
Предсказанный класс: нейтральный
Вероятности: {'нейтральный': '0.7606', 'позитивный': '0.2256', 'негативный': '0.0138'}
----------------------------------------

Текст: Супер! Рекомендую всем!
Предсказанный класс: позитивный
Вероятности: {'нейтральный': '0.0044', 'позитивный': '0.9938', 'негативный': '0.0018'}
--------------------------

In [19]:
import joblib
import os

os.makedirs('models', exist_ok=True)

joblib.dump(model, 'models/sentiment_model.pkl')
joblib.dump(vectorizer, 'models/tfidf_vectorizer.pkl')
joblib.dump(preprocess_text, 'models/preprocessor.pkl')

print("\n" + "="*60)
print("Модель и векторизатор сохранены в папке 'models/'")
print("="*60)

def load_and_predict():

    loaded_model = joblib.load('models/sentiment_model.pkl')
    loaded_vectorizer = joblib.load('models/tfidf_vectorizer.pkl')
    loaded_preprocessor = joblib.load('models/preprocessor.pkl')
    
    while True:
        user_text = input("\nВведите текст для анализа (или 'выход' для завершения): ")
        if user_text.lower() == 'выход':
            break
        
        result = predict_sentiment(user_text, loaded_model, loaded_vectorizer)
        print(f"\nАнализ текста:")
        print(f"Класс: {result['prediction']}")
        print(f"Вероятности: {result['probabilities']}")

print("\n" + "="*60)
print("КРОСС-ВАЛИДАЦИЯ:")
print("="*60)

from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
print(f"Accuracy при 5-кратной кросс-валидации: {cv_scores.mean():.4f} (+/- {cv_scores.std()*2:.4f})")


Модель и векторизатор сохранены в папке 'models/'

КРОСС-ВАЛИДАЦИЯ:
Accuracy при 5-кратной кросс-валидации: 0.6453 (+/- 0.1232)
